In [1]:
import os

In [2]:
%pwd

'/Users/kavehnamvar/Documents/learn to program/End to end projects/Chicken-disease-classification/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/kavehnamvar/Documents/learn to program/End to end projects/Chicken-disease-classification'

first we need to write the config.yaml file

In [5]:
#initialize the entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

    

In [6]:
# now create the configuration manager
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
    

In [8]:
# now create the component
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [11]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    

    @property
    def _create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir, 
            f"tb_logs_at_(timestamp)",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callback, self._create_ckpt_callbacks]

In [12]:
#  prepare the pipeline
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[2025-02-07 00:44:21,935: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-02-07 00:44:21,939: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-07 00:44:21,940: INFO: common: created directory at: artifacts]
[2025-02-07 00:44:21,941: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2025-02-07 00:44:21,942: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]


In [14]:
#test 
import time
timsstamp = time.strftime("%Y-%m-%d_%H-%M-%S")
f"tb_logs_at_{timsstamp}"


'tb_logs_at_2025-02-07_00-46-24'

now update entity: config_entity.py
then update the configurationmanager: configuration.py
now update components
    to do this go inside components and create a file prepare_callbacks.py
here we don't need to create the pipeline
now start implementing the next component which is model trainer